# This is the first attempt at solving the problem #

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [ ]:
train = pd.read_json('../input/train.json')
train.head(20)

## Missing Values ##

In [ ]:
missing = train.isnull().sum()
missing

No missing values

In [ ]:
idx = train['features'].index
feat_count = list()
feat_count += [len(train.features[i]) for i in idx]
train['feat_count'] = feat_count

In [ ]:
from nltk.corpus import sentiwordnet as swn
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from collections import Counter
import nltk
import string
import re

In [ ]:
def polarity (data, text_col):
    
#    print ('Calculating polarities.....')    
    
    sw = stopwords.words('english')
    
    p_c = []
    n_c = []

    pos_polarity = []
    neg_polarity = []

    for i in range(0, data.shape[0]):
    
        pos_count = 0
        neg_count = 0
        
        pos_score = 0
        neg_score = 0

        for w in word_tokenize(data.iloc[i][text_col].decode('utf-8')):
            if w not in sw:
                try:
                    sent = list(swn.senti_synsets(w))[0]
                    pc = sent.pos_score()
                    nc = sent.neg_score()
                    if (pc > nc):
                        pos_count += 1
                        pos_score += pc
                    elif(pc < nc):
                        neg_count += 1
                        neg_score += nc
                except:
                    pass
        
        p_c.append(pos_count)
        n_c.append(neg_count)
        
        pos_polarity.append(pos_score)
        neg_polarity.append(neg_score)      
        
    data['n_count'] = n_c
    data['p_count'] = p_c
    
    data['n_pol'] = neg_polarity
    data['p_pol'] = pos_polarity
    
    data.loc[data.n_pol > data.p_pol, 'pol'] = 0
    data.loc[data.n_pol < data.p_pol, 'pol'] = 1
    data.loc[data.n_pol == data.p_pol, 'pol'] = 2
    
    data['pol_ratio'] = (data.p_pol + 0.5) / (data.n_pol + 0.5) 

In [ ]:
train.head()

In [ ]:
X = train[['bedrooms', 'bathrooms', 'feat_count']]
y = train.interest_level

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

X_train, X_cv, y_train, y_cv = train_test_split(X, y, test_size=0.3)
rf = RandomForestClassifier(n_estimators=25)
nb = MultinomialNB()
rf.fit(X_train, y_train)
nb.fit(X_train, y_train)

In [ ]:
y_pred_nb = nb.predict(X_cv)
y_pred_rf = rf.predict(X_cv)

from sklearn.metrics import confusion_matrix, accuracy_score
sns.heatmap(confusion_matrix(y_cv, y_pred_nb), annot=True, fmt='1g')
print('Accuracy : ', 
     accuracy_score(y_cv, y_pred_nb))

In [ ]:
sns.heatmap(confusion_matrix(y_cv, y_pred_rf), annot=True, fmt='1g')
print('Accuracy : ', 
     accuracy_score(y_cv, y_pred_rf))